In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=d3bb2ea602059e2e2894869e32e903ac2d81ac0deb676925a7dd71ab77e4b05b
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install kaggle nbdev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.7 MB/s eta 0:00:00


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName("exemplo bla bla").config("spark.driver.memory","4g").config("spark.executor.memory","4g").getOrCreate()

In [6]:
# Some general imports 

import os
import sys

import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [7]:
def plotHistogram(df, xcol, huecol=None):
    sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")

In [8]:
def plot(df, xcol, ycol):
    sns.lineplot(data=df, x=xcol, y=ycol)

In [9]:
def plotBar(df, xcol, ycol, huecol=None):
    sns.barplot(data=df, x=xcol, y=ycol, hue=huecol)

In [10]:
def plotScatter(df, xcol, ycol, huecol=None):
    sns.scatterplot(data=df, x=xcol, y=ycol, hue=huecol)

In [11]:
def plotScatterMatrix(df, huecol=None):
    sns.pairplot(data=df, hue=huecol)

In [12]:
def plotCorrelationMatrix_1(df, annot=False):
    # compute the correlation matrix
    corr = df.corr()
    
    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    #cmap='coolwarm'

    # draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=annot,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [13]:
def plotCorrelationMatrix_2(df):
    # compute a correlation matrix and convert to long-form
    corr_mat = df.corr().stack().reset_index(name="correlation")
    # draw each cell as a scatter point with varying size and color
    g = sns.relplot(
        data=corr_mat,
        x="level_0", y="level_1", hue="correlation", size="correlation",
        palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),
    )

    # tweak the figure to finalize
    g.set(xlabel="", ylabel="", aspect="equal")
    g.despine(left=True, bottom=True)
    g.ax.margins(.02)
    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

In [14]:
# Some Spark related imports we will use hereafter

import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline, Transformer
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, SQLTransformer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import row_number, monotonically_increasing_id, split, transform
from pyspark.sql import Window



In [ ]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [16]:
train_schema = StructType([StructField('session_id', LongType(), True),
                     StructField('index', IntegerType(), True),
                     StructField('elapsed_time', IntegerType(), True),
                     StructField('event_name', StringType(), True),                  
                     StructField('name', StringType(), True),      
                     StructField('level', IntegerType(), True),                
                     StructField('page', DoubleType(), True),
                     StructField('room_coor_x', DoubleType(), True),
                     StructField('room_coor_y', DoubleType(), True),
                     StructField('screen_coor_x', DoubleType(), True),
                     StructField('screen_coor_y', DoubleType(), True),
                     StructField('hover_duration', DoubleType(), True),
                     StructField('text', StringType(), True),
                     StructField('fqid', StringType(), True),
                     StructField('room_fqid', StringType(), True),
                     StructField('text_fqid', StringType(), True),       
                     StructField('fullscreen', StringType(), True),       
                     StructField('hq', IntegerType(), True),       
                     StructField('music', IntegerType(), True),                 
                     StructField('level_group', IntegerType(), True),       
                     StructField('session_level', StringType(), True)])

filename = "train.csv"
df_train = spark.read.csv(filename, header=True, schema=train_schema, sep=',')


AnalysisException: ignored

In [15]:
test_schema = StructType([StructField('session_id', LongType(), True),
                     StructField('index', IntegerType(), True),
                     StructField('elapsed_time', IntegerType(), True),
                     StructField('event_name', StringType(), True),                  
                     StructField('name', StringType(), True),      
                     StructField('level', IntegerType(), True),                
                     StructField('page', DoubleType(), True),
                     StructField('room_coor_x', DoubleType(), True),
                     StructField('room_coor_y', DoubleType(), True),
                     StructField('screen_coor_x', DoubleType(), True),
                     StructField('screen_coor_y', DoubleType(), True),
                     StructField('hover_duration', DoubleType(), True),
                     StructField('text', StringType(), True),
                     StructField('fqid', StringType(), True),
                     StructField('room_fqid', StringType(), True),
                     StructField('text_fqid', StringType(), True),       
                     StructField('fullscreen', StringType(), True),       
                     StructField('hq', IntegerType(), True),       
                     StructField('music', IntegerType(), True),                 
                     StructField('level_group', IntegerType(), True),       
                     StructField('session_level', StringType(), True)])

filename = "test.csv"
df_teste = spark.read.csv(filename, header=True, schema=test_schema, sep=',')

In [ ]:
train_labs_schema = StructType([StructField('session_id', StringType(), True),
                     StructField('correct', IntegerType(), True)])

filename = "train_labels.csv"
df_train_labs = spark.read.csv(filename, header=True, schema=train_labs_schema, sep=',')

In [ ]:
sample_sub_schema = StructType([StructField('session_id', StringType(), True),
                     StructField('correct', IntegerType(), True)])

filename = "sample_submission.csv"
df_sample_sub = spark.read.csv(filename, header=True, schema=sample_sub_schema, sep=',')

In [18]:
df_sample_sub.printSchema()

NameError: ignored

In [ ]:
df_train.select("*").show(12)

+-----------------+-----+------------+--------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+--------------------+------+--------------------+--------------------+----------+---+-----+-----------+-------------+
|       session_id|index|elapsed_time|    event_name|     name|level|page|        room_coor_x|        room_coor_y|screen_coor_x|screen_coor_y|hover_duration|                text|  fqid|           room_fqid|           text_fqid|fullscreen| hq|music|level_group|session_level|
+-----------------+-----+------------+--------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+--------------------+------+--------------------+--------------------+----------+---+-----+-----------+-------------+
|20090312431273200|    0|           0|cutscene_click|    basic|    0|null|-413.99140522263224|-159.31468617022784|        380.0|        494.0|          null|           undefin

In [ ]:
# df_teste2 = df_teste.withColumn(
#     "id_new",
#     row_number().over(Window.orderBy(monotonically_increasing_id()))-1
# )

In [ ]:
# df_teste2.toPandas()

In [ ]:
# train_schema = StructType([StructField('session_id', LongType(), True),
#                      StructField('index', IntegerType(), True),
#                      StructField('elapsed_time', IntegerType(), True),
#                      StructField('event_name', StringType(), True),                  
#                      StructField('name', StringType(), True),      
#                      StructField('level', IntegerType(), True),                
#                      StructField('page', DoubleType(), True),
#                      StructField('room_coor_x', DoubleType(), True),
#                      StructField('room_coor_y', DoubleType(), True),
#                      StructField('screen_coor_x', DoubleType(), True),
#                      StructField('screen_coor_y', DoubleType(), True),
#                      StructField('hover_duration', DoubleType(), True),
#                      StructField('text', StringType(), True),
#                      StructField('fqid', StringType(), True),
#                      StructField('room_fqid', StringType(), True),
#                      StructField('text_fqid', StringType(), True),       
#                      StructField('fullscreen', StringType(), True),       
#                      StructField('hq', IntegerType(), True),       
#                      StructField('music', IntegerType(), True),                 
#                      StructField('level_group', IntegerType(), True),       
#                      StructField('session_level', StringType(), True)])

In [ ]:
df_teste.select("*").show(12)

+-----------------+-----+------------+-----------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+--------------------+------+--------------------+--------------------+----------+---+-----+-----------+-------------+
|       session_id|index|elapsed_time|       event_name|     name|level|page|        room_coor_x|        room_coor_y|screen_coor_x|screen_coor_y|hover_duration|                text|  fqid|           room_fqid|           text_fqid|fullscreen| hq|music|level_group|session_level|
+-----------------+-----+------------+-----------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+--------------------+------+--------------------+--------------------+----------+---+-----+-----------+-------------+
|20090109393214576|    0|           0|   cutscene_click|    basic|    0|null|-413.99140522263224|   75.6853138297722|        380.0|        259.0|          null|      

In [ ]:
df_todo = df_train.union(df_teste)


In [26]:
df_teste = df_teste.withColumn(
    "id_new",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)

In [19]:
def time_paseed(a):
    kard = a.collect()
    lss = 0
    lastos = []
    for i in kard:
        try:
            lastos.append(kard[lss+1][0] - i[0])
        except IndexError:
            lastos.append(0)
        # print(kard[lss+1][0]-i[0])
        # print(kard[lss+1][0])
        lss+= 1
    return lastos
        
        

In [21]:
def converto(karr):
  lasldlasd = time_paseed(karr.select("elapsed_time"))
  labels_udf = F.udf(lambda indx: lasldlasd[indx-1], IntegerType())
  karr = karr.withColumn("nummm_id", row_number().over(Window.orderBy(monotonically_increasing_id())))
  karr = karr.withColumn('time_passed', labels_udf('nummm_id'))
  karr = karr.drop('nummm_id')
  return karr

In [26]:
df_teste.select("elapsed_time","time_passed").toPandas()

,elapsed_time,time_passed
0,0,1965
1,1965,1649
2,3614,1716
3,5330,1067
4,6397,2467
...,...,...
3723,1581679,1365
3724,1583044,366
3725,1583410,2431
3726,1585841,1969


In [22]:
def time_paseed_group(a):
    kard = a.collect()
    lss = 0
    lastos = []
    for i in kard:
        # print(i[1])
        try:
            lastos.append([i[0],kard[lss+1][1] - i[1]])
        except IndexError:
            lastos.append([i[0],lss-lss])
        # #print(kard[1][lss+1]-i[1])
        # print(kard[lss+1][1])
        lss+= 1
    return lastos
        

In [110]:
groupss = df_teste.select("session_id","event_name").distinct().collect()
len(groupss)
df_teste.count()

3728

In [93]:
time_paseed_group(df_teste.filter(f"session_id == {i[0]}" and f"event_name == '{i[1]}'").select("id_new","elapsed_time"))

17780
19980


[]

In [ ]:
df_teste.select("session_id","event_name").distinct().toPandas()

In [154]:
def func_groups(xa):
  emp_RDD = spark.sparkContext.emptyRDD()
  columns = StructType([StructField('session_id', LongType(), True),
                      StructField('index', IntegerType(), True),
                      StructField('elapsed_time', IntegerType(), True),
                      StructField('event_name', StringType(), True),                  
                      StructField('name', StringType(), True),      
                      StructField('level', IntegerType(), True),                
                      StructField('page', DoubleType(), True),
                      StructField('room_coor_x', DoubleType(), True),
                      StructField('room_coor_y', DoubleType(), True),
                      StructField('screen_coor_x', DoubleType(), True),
                      StructField('screen_coor_y', DoubleType(), True),
                      StructField('hover_duration', DoubleType(), True),
                      StructField('text', StringType(), True),
                      StructField('fqid', StringType(), True),
                      StructField('room_fqid', StringType(), True),
                      StructField('text_fqid', StringType(), True),       
                      StructField('fullscreen', StringType(), True),       
                      StructField('hq', IntegerType(), True),       
                      StructField('music', IntegerType(), True),                 
                      StructField('level_group', StringType(), True),       
                      StructField('session_level', StringType(), True),
                      StructField('id_new', IntegerType(), True),
                      StructField('time_passed_grups', IntegerType(), True)])
  kakakka = spark.createDataFrame(data = emp_RDD,schema = columns)
  itera = 0
  joinDF2 = []
  groupss = xa.select("session_id","event_name").distinct().collect()
  for i in groupss:
    arraysalvo = time_paseed_group(xa.filter(f"session_id == {i[0]}" and f"event_name == '{i[1]}'").select("id_new","elapsed_time"))
    labels_array = F.udf(lambda indx: arraysalvo[indx-1][1], IntegerType())
    df_teste_12 = xa.filter(f"session_id == {i[0]}" and f"event_name == '{i[1]}'").withColumn("id_groups", row_number().over(Window.orderBy(monotonically_increasing_id())))
    df_teste_12 = df_teste_12.withColumn('time_passed_grups', labels_array('id_groups'))
    df_teste_12 = df_teste_12.drop('id_groups')
    kakakka.createOrReplaceTempView("fixa")
    df_teste_12.createOrReplaceTempView("tempo")
    kakakka = spark.sql("SELECT * FROM tempo UNION SELECT * FROM fixa")
  return kakakka


In [155]:
asdddd = func_groups(df_teste)

In [156]:
asdddd.count()

3728

In [148]:
from pyspark.ml.param import P
# Create an empty RDD
emp_RDD = spark.sparkContext.emptyRDD()
 
# Create empty schema
columns = StructType([StructField('session_id', LongType(), True),
                     StructField('index', IntegerType(), True),
                     StructField('elapsed_time', IntegerType(), True),
                     StructField('event_name', StringType(), True),                  
                     StructField('name', StringType(), True),      
                     StructField('level', IntegerType(), True),                
                     StructField('page', DoubleType(), True),
                     StructField('room_coor_x', DoubleType(), True),
                     StructField('room_coor_y', DoubleType(), True),
                     StructField('screen_coor_x', DoubleType(), True),
                     StructField('screen_coor_y', DoubleType(), True),
                     StructField('hover_duration', DoubleType(), True),
                     StructField('text', StringType(), True),
                     StructField('fqid', StringType(), True),
                     StructField('room_fqid', StringType(), True),
                     StructField('text_fqid', StringType(), True),       
                     StructField('fullscreen', StringType(), True),       
                     StructField('hq', IntegerType(), True),       
                     StructField('music', IntegerType(), True),                 
                     StructField('level_group', StringType(), True),       
                     StructField('session_level', StringType(), True),
                     StructField('id_new', IntegerType(), True),
                     StructField('time_passed_grups', IntegerType(), True)])
 
# Create an empty RDD with empty schema
kakakka = spark.createDataFrame(data = emp_RDD,schema = columns)
aaa= 0
for i in asdddd:
  kakak = i.select("id_new").collect()
  break
  print(i.select("id_new").collect() not in kakakka.select("id_new").collect())
  if i.select("session_id", "event_name").collect() not in kakakka.select("session_id", "event_name").collect():
    #kakakka = kakakka.union(i)\
    pass
  #i.show()
  

In [150]:
len(kakak)

1519

In [138]:
kakakka.count()

11184

In [165]:
df_teste_12.show()

+-----------------+-----+------------+--------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+----+----------+--------------------+---------+----------+---+-----+-----------+-------------+-----------+------+---------+-----------------+
|       session_id|index|elapsed_time|    event_name|     name|level|page|        room_coor_x|        room_coor_y|screen_coor_x|screen_coor_y|hover_duration|text|      fqid|           room_fqid|text_fqid|fullscreen| hq|music|level_group|session_level|time_passed|id_new|id_groups|time_passed_grups|
+-----------------+-----+------------+--------------+---------+-----+----+-------------------+-------------------+-------------+-------------+--------------+----+----------+--------------------+---------+----------+---+-----+-----------+-------------+-----------+------+---------+-----------------+
|20090109393214576|   10|       17780|navigate_click|undefined|    0|null|  534.1177123473617| -92.7487

In [59]:
b = f"{20090312331414616}"
a = "session_id == {}".format(b)
b1 = "observation_click"
a1 = f"event_name == '{b1}'"
df_teste.filter(a).filter(a1).count()

8

In [69]:
df_teste.filter("session_id == {}".format() and "event_name == {}".format()).select("elapsed_time").count()

965

In [70]:
asdwd = time_paseed_group(df_teste.filter("session_id == 20090312331414616").select("elapsed_time"))

In [71]:
len(asdwd)

965

In [ ]:
df_train = df_train.withColumn(
    "time_passed_group",
    time_paseed(df_train.select("elapsed_time"))
)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\anaconda\instalation\envs\pbd\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\anaconda\instalation\envs\pbd\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\anaconda\instalation\envs\pbd\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\anaconda\instalation\envs\pbd\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [ ]:
df_train = df_train.withColumn(
    "time_group",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)

In [ ]:
df_train_labs.select(split("session_id", '_',-1).getItem(0).alias('session_id'),split("session_id", '_',-1).getItem(1).alias('question'),'correct').show(12)

+-----------------+--------+-------+
|       session_id|question|correct|
+-----------------+--------+-------+
|20090312431273200|      q1|      1|
|20090312433251036|      q1|      0|
|20090312455206810|      q1|      1|
|20090313091715820|      q1|      0|
|20090313571836404|      q1|      1|
|20090314035813970|      q1|      1|
|20090314121766812|      q1|      1|
|20090314221187252|      q1|      0|
|20090314363702160|      q1|      1|
|20090314441803444|      q1|      1|
|20090315081004164|      q1|      0|
|20090315085850788|      q1|      1|
+-----------------+--------+-------+
only showing top 12 rows



In [ ]:
df_train_labs = df_train_labs.select(split("session_id", '_',-1).getItem(0).alias('session_id'),split("session_id", '_',-1).getItem(1).alias('question'),'correct')

In [ ]:
df_train_labs.show(5)

+-----------------+--------+-------+
|       session_id|question|correct|
+-----------------+--------+-------+
|20090312431273200|      q1|      1|
|20090312433251036|      q1|      0|
|20090312455206810|      q1|      1|
|20090313091715820|      q1|      0|
|20090313571836404|      q1|      1|
+-----------------+--------+-------+
only showing top 5 rows



In [ ]:
df_train_labs = df_train_labs.withColumn("session_id",df_train_labs["session_id"].cast('Long'))

In [ ]:
df_train_labs.toPandas()

,session_id,question,correct
0,20090312431273200,q1,1
1,20090312433251036,q1,0
2,20090312455206810,q1,1
3,20090313091715820,q1,0
4,20090313571836404,q1,1
...,...,...,...
424111,22100215342220508,q18,1
424112,22100215460321130,q18,1
424113,22100217104993650,q18,1
424114,22100219442786200,q18,1


In [ ]:
df_train_labs.show(12).groupby('session_id')

In [ ]:
#df_todo.write.mode("overwrite").parquet("all_data_predict")

In [ ]:
df_train_labs.select("session_id").toPandas()

In [ ]:
#colunios = split(df_train_labs.select('session_id'), '_')

In [ ]:
df_todo.select("session_id").distinct().toPandas()

In [ ]:
df_train_labs.select("session_id").distinct().toPandas()

In [ ]:
23565/18

1309.1666666666667

In [ ]:
listaasd = df_train_labs.select("session_id").distinct().collect()

In [ ]:
listaasd

In [ ]:
asd = df_train.select("session_id").distinct().collect()

In [ ]:
asd

In [ ]:
1435*23562

33811470

In [ ]:
kar = [x for x in asd if x in listaasd]

In [ ]:
len(kar)

0

In [ ]:
df_train_labs.select("session_id"==listaasd).show()

In [ ]:
#df_todo = df_todo.select("*").withColumn("id_index", monotonically_increasing_id())

In [ ]:
#df_todo.select("*").toPandas()

In [ ]:
df_teste.show(1, vertical=True)
teste_count = df_teste.count()
df_teste.show(1)

-RECORD 0------------------------------
 session_id     | 20090109393214576    
 index          | 0                    
 elapsed_time   | 0                    
 event_name     | cutscene_click       
 name           | basic                
 level          | 0                    
 page           | null                 
 room_coor_x    | -413.99140522263224  
 room_coor_y    | 75.6853138297722     
 screen_coor_x  | 380.0                
 screen_coor_y  | 259.0                
 hover_duration | null                 
 text           | undefined            
 fqid           | intro                
 room_fqid      | tunic.historicals... 
 text_fqid      | tunic.historicals... 
 fullscreen     | null                 
 hq             | null                 
 music          | null                 
 level_group    | 0-4                  
 session_level  | 20090109393214576... 
only showing top 1 row

+-----------------+-----+------------+--------------+-----+-----+----+-------------------+------

In [ ]:
df_train.show(1, vertical=True)
train_count = df_train.count()
df_train.show(1)

-RECORD 0------------------------------
 session_id     | 20090312431273200    
 index          | 0                    
 elapsed_time   | 0                    
 event_name     | cutscene_click       
 name           | basic                
 level          | 0                    
 page           | null                 
 room_coor_x    | -413.99140522263224  
 room_coor_y    | -159.31468617022784  
 screen_coor_x  | 380.0                
 screen_coor_y  | 494.0                
 hover_duration | null                 
 text           | undefined            
 fqid           | intro                
 room_fqid      | tunic.historicals... 
 text_fqid      | tunic.historicals... 
 fullscreen     | null                 
 hq             | null                 
 music          | null                 
 level_group    | 0-4                  
only showing top 1 row

+-----------------+-----+------------+--------------+-----+-----+----+-------------------+-------------------+-------------+------------

In [ ]:
seed = 10
with_replacement = False
fraction = 0.5          # reduce to 20%
small_df_train = df_train.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)

In [ ]:
small_df_train.select("*").count()

13148649

In [ ]:
small_df_train.groupby("session_id","index").count().sort(F.col("count").desc()).show(337)

In [ ]:
df_train.groupby("session_id","index").count().filter("count >= 2").groupby("session_id").count().show()

In [ ]:
asd = df_train.filter("session_id==21000108034275240")

In [ ]:
asd.write.csv("kawkdakwd")

In [ ]:
df_teste.toPandas()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0.0,1.0,NaN,0
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0.0,1.0,NaN,0
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0.0,1.0,NaN,0
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0.0,1.0,NaN,0
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0.0,1.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,...,484.0,None,tunic.wildlife,tunic.historicalsociety.entry,None,0,0.0,1.0,NaN,2
3724,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,...,783.0,None,tunic.capitol_2,tunic.historicalsociety.entry,None,0,0.0,1.0,NaN,2
3725,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726363,-3.880047,456.0,...,NaN,None,tunic.capitol_2,tunic.historicalsociety.entry,None,0,0.0,1.0,NaN,2
3726,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372139,38.216178,383.0,...,NaN,None,chap4_finale_c,tunic.capitol_2.hall,None,0,0.0,1.0,NaN,2


In [ ]:
type(df_teste)

pyspark.sql.dataframe.DataFrame

In [ ]:
def trasnfo(x):
    return upper(x)

In [ ]:
df_teste.select(transform("text", trasnfo)).show(10)

NameError: name 'upper' is not defined

In [ ]:
df_teste.select("name","session_id",transform("text", lambda x: upper(x)).alias("text")).show(10)

NameError: name 'upper' is not defined

In [ ]:
sparkContext=spark.sparkContext

In [ ]:
rdd=sparkContext.parallelize([1,2,3,4,5])
rddCollect = rdd.collect()
print("Number of Partitions: "+str(rdd.getNumPartitions()))
print("Action: First element: "+str(rdd.first()))
print(rddCollect)

Number of Partitions: 4


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 5) (DESKTOP-9NRQ44U executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\anaconda\instalation\envs\pbd\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 540, in main
RuntimeError: Python in worker has different version 3.9 than that in driver 3.10, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:552)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:758)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:

In [ ]:
sessions = df_train.groupby('session_id') \
    .agg(
        max(F.col("index")).alias("max_index"),
        
    ).withColumn(
    "id",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)
sessions.limit(10)

TypeError: Column is not iterable

In [ ]:
cls_not_features = ['Time', 'Merchant State', 'Zip', 'Is Fraud?', 'Fraud'] 
cls_numeric = ['Time', 'Merchant State', 'Zip', 'Is Fraud?', 'Fraud']
cls_non_numeric = [i for i in transactions_cols_interest if i not in cls_numeric]
cls_non_numeric

SyntaxError: invalid syntax (958298010.py, line 2)

In [ ]:
# Encoding columns and vector assembling them
# See Chapter 10 of the book "Learning Spark - Lightning-Fast Data Analytics"

categorical_cols = [i for i in cls_non_numeric if i not in cls_not_features]
non_categorical_cols = [i for i in cls_numeric if i not in cls_not_features]

index_output_cols = [x + ' Index' for x in categorical_cols]
ohe_output_cols = [x + ' OHE' for x in categorical_cols]

string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=index_output_cols, handleInvalid="skip")

ohe_encoder = OneHotEncoder(inputCols=index_output_cols, outputCols=ohe_output_cols)
   
# Put all input features into a single vector, by using a transformer

assembler_inputs = ohe_output_cols + non_categorical_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
assembler_inputs

In [ ]:
pipeline = Pipeline(stages=[string_indexer, ohe_encoder, vec_assembler, lsvc])

NameError: name 'string_indexer' is not defined

In [ ]:
sqlTrans.getParam

<bound method Params.getParam of SQLTransformer_0ce761cbfe95>

In [ ]:
cols_to_forget = ['session_id']
cards_cols_interest = [x for x in df_teste.columns if x in cols_to_forget]

In [ ]:
df_teste.describe(cards_cols_interest).show()

In [ ]:
'session_id == 20100308381327090'

In [ ]:
df_aaa = ( df_teste
                .withColumn("session_id", F.when(F.col('session_id')=='20100308381327090', 1).otherwise(0))
            )
df_aaa

In [ ]:
df_teste.filter(F.expr('session_id == 20090312331414616')).toPandas()

In [ ]:
df_teste.where('session_id == 20100308381327090').show()

In [ ]:
#df_teste.groupBy('level').count().toPandas().sort(df_teste.col("level"),("count")).show(truncate=False)

In [ ]:
df_teste.groupBy('session_id').count().toPandas()


#incialmente pensamento em utilziar o dataset de teste para trabalhar com um dataset menor, foi verificado que no dataset de teste possui apenas 3 session ID unicos enquanto que no dataset de treino tem 11779 session id unicos por isso não podemos utilziar o dataset de teste, teremos de fazer uma reduçao apartir dp dataset de treino

In [ ]:
df_teste.groupBy('text').count().toPandas()

In [ ]:
df_teste.groupBy('event_name').count().toPandas()

In [ ]:
df_plot = df_teste.groupBy('event_name').count().toPandas()
plotBar(df_plot, 'event_name', 'count')
plt.title('grupos (event_name)')
plt.show()

In [ ]:
df_train[f.Column'session_id'].printSchema()
df_train.show(1, vertical=True)
train_count = df_train.count()
train_count
df_train.show(1)

In [ ]:
df_train_labs.show(5)

In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['Credit Limit']
teste_cols_interest = [x for x in df_teste.columns]
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')


Nulls in all columns


NameError: name 'teste_count' is not defined

In [ ]:
test

In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['fullscreen','hq','music','hover_duration']
train_cols_interest = [x for x in df_train.columns if df_train.columns not in cols_to_forget]
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')

In [ ]:
df_train.head(4)

In [ ]:
df_train.tail(4)

In [ ]:
train_cols_interest.remove('fullscreen')
train_cols_interest.remove('music')
train_cols_interest.remove('hq')
train_cols_interest.remove('page')
train_cols_interest.remove('hover_duration')
train_cols_interest.remove('text')
train_cols_interest.remove('text_fqid')

In [ ]:
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')

In [ ]:
teste_cols_interest.remove('fullscreen')
teste_cols_interest.remove('music')
teste_cols_interest.remove('hq')
teste_cols_interest.remove('page')

In [ ]:
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')

In [ ]:
print('\nUniqueness in teste:')
for cl in teste_cols_interest:
    k = df_teste.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {teste_count}"+f" records ({k*100/teste_count:.2f})%")

In [ ]:
print('\nUniqueness in treino:')
for cl in train_cols_interest:
    k = df_train.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {train_count}"+f" records ({k*100/train_count:.2f})%")

In [ ]:
#corr_matrix = Correlation.corr(df_teste, df_teste).collect()[0][0].toArray().tolist()

In [ ]:
#df_plot = pd.DataFrame(data = corr_matrix, index=df.teste.columns, columns=df.teste.columns)
#plotCorrelationMatrix_1(df_plot, annot=True)
#plt.title('Correlations among numerical features')
#plt.show()

In [ ]:
seed = 5
with_replacement = False
fraction = 0.2          # reduce to 20%
small_df_train = df_train.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)

In [ ]:
small_df_train.count()

In [ ]:
df_train.count()

In [ ]:
small_df_train.write.mode("overwrite").parquet("small-df_train")

In [ ]:
df_train_clean = spark.read.parquet('small-df_train')

In [ ]:
df_train_clean.toPandas()

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,21050410434308160,968,1726999,person_click,basic,20,NaN,-381.847136,110.712177,254.0,245.0,NaN,It has something to do with ecology.,worker,tunic.library.frontdesk,tunic.library.frontdesk.worker.flag,None,None,None,13-22
1,21050410434308160,978,1735199,navigate_click,undefined,20,NaN,586.732639,227.529868,762.0,155.0,NaN,None,tomicrofiche,tunic.library.frontdesk,None,None,None,None,13-22
2,21050410434308160,985,1754449,object_click,basic,20,NaN,32.837203,-229.000000,480.0,559.0,NaN,None,reader_flag.paper1.next,tunic.library.microfiche,None,None,None,None,13-22
3,21050410434308160,986,1758732,object_hover,undefined,20,NaN,NaN,NaN,NaN,NaN,10699.0,None,reader_flag.paper0.next,tunic.library.microfiche,None,None,None,None,13-22
4,21050410434308160,987,1759332,object_hover,undefined,20,NaN,NaN,NaN,NaN,NaN,566.0,None,reader_flag.paper2.bingo,tunic.library.microfiche,None,None,None,None,13-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634645,22100221145014656,1570,5431389,object_click,basic,21,NaN,-257.274472,45.755580,107.0,286.0,NaN,None,journals_flag.hub.topics,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634646,22100221145014656,1584,5445520,object_hover,undefined,21,NaN,NaN,NaN,NaN,NaN,404.0,None,journals_flag.pic_1.next,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634647,22100221145014656,1586,5447903,object_click,basic,21,NaN,447.725528,-72.244420,812.0,404.0,NaN,None,journals_flag.pic_2.next,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634648,22100221145014656,1590,5470870,navigate_click,undefined,22,NaN,10.725528,-282.244420,375.0,614.0,NaN,None,toentry,tunic.historicalsociety.stacks,None,None,None,None,13-22


In [ ]:
df_train_on = df_train.select(train_cols_interest)
df_teste_on = df_teste.select(teste_cols_interest)

In [ ]:
df_train_on.printSchema()
df_teste_on.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- level_group: string (nullable = true)

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable =